In [ ]:
import collections
import json
import random
import re
from itertools import chain
from multiprocessing import Pool
from pathlib import Path
from typing import *

import git
import pandas as pd
import pydriller
from pymongo import MongoClient

In [ ]:
import dotenv
dotenv.load_dotenv()
ENV = dotenv.dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])
MVN_ECO_DIR = Path(ENV["MVN_ECO_DIR"])
DATA_DIR, DATA_DIR.exists(), MVN_ECO_DIR, MVN_ECO_DIR.exists()

In [ ]:
mvn_eco_dep_data = pd.read_csv(MVN_ECO_DIR/'csv'/'DEP.csv').to_dict(orient='records')
mvn_eco_patch_data = pd.read_csv(MVN_ECO_DIR/'csv'/'PATCH.csv').to_dict(orient='records')
mvn_eco_cve_data = pd.read_csv(MVN_ECO_DIR/'csv'/'CVE.csv').to_dict(orient='records')
mvn_eco_dep_data[0].keys(), mvn_eco_patch_data[0].keys(), mvn_eco_cve_data[0].keys()

In [ ]:
len(list(x['CVE_ID'] for x in mvn_eco_cve_data)), len(set(x['CVE_ID'] for x in mvn_eco_cve_data)), len(list(x['CVE_ID'] for x in mvn_eco_patch_data)), len(set(x['CVE_ID'] for x in mvn_eco_patch_data))

In [ ]:
mvn_eco_cve_dict = dict([(x['CVE_ID'], x) for x in mvn_eco_cve_data])
mvn_eco_patch_dict = dict([(x['CVE_ID'], x) for x in mvn_eco_patch_data])
len(set(mvn_eco_cve_dict.keys()).intersection(set(mvn_eco_patch_dict.keys())))

In [ ]:
records = list()
for cve_id in mvn_eco_cve_dict.keys():
    vuln_gav = mvn_eco_patch_dict[cve_id]['G:A:V']
    vuln_funs: list[str] = mvn_eco_cve_dict[cve_id]['VUL_FUNs'].split(";")
    vuln_classes = set()
    vuln_methods = set()
    for vf in vuln_funs:
        vc = vf.split(':')[0].strip()
        vc = vc.split('$')[0]
        
        vm = vf.split(':')[1].split('(')[0].strip()
        
        if vm == '<init>':
            vm = vc.split('.')[-1].strip()
        
        vm = vc.split('.')[-1].strip() + '::' + vm
        
        vuln_classes.add(vc)
        vuln_methods.add(vm)
    
    r = dict()
    r['data_src'] = 'mvn_eco'
    r['cve_ref'] = cve_id
    r['vuln_gav'] = vuln_gav
    r['vuln_classes'] = list(vuln_classes)
    # r['vuln_methods'] = list(vuln_methods)
    r['patches'] = mvn_eco_patch_dict[cve_id]['PATCH'].split(';')
    
    records.append(r)

In [ ]:
random.choice(records)

In [ ]:
client = MongoClient("localhost", 42692)
db = client.s5_snyk_libio
db.mvnEcoVuln.drop()
_ = db.mvnEcoVuln.insert_many(records)